### Библиотеки / данные

импортируем библиотеки numpy и pandas

In [ ]:
import numpy as np
import pandas as pd

загружаем данные

In [ ]:
!gdown 16MSmqLRcyq3iIAkO4U2i6xB1en4-ZlV2

In [ ]:
tips = pd.read_csv(filepath_or_buffer = "tips.csv")
tips.head()

In [ ]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [ ]:
tips.head()

### Механизм GroupBy

In [ ]:
df = pd.DataFrame({'x': ['a','a','b','b','c','c'],
                   'y': [2,4,0,5,5,10]})
df

In [ ]:
groups = df.groupby(['x'])
groups

получаем информацию о количестве групп, которые будут созданы

In [ ]:
groups.ngroups

получаем информацию о количестве элементов в каждой группе

In [ ]:
groups.size()

что представляют из себя найденные группы?

In [ ]:
groups.groups

получаем данные конкретной группы

In [ ]:
groups.get_group('b')

извлекаем первую строку каждой группы

In [ ]:
groups.nth([1])

обход групп:

In [ ]:
for key, group in groups:
    print(key)
    print(group) 

вычисление среднего

In [ ]:
groups.y.mean()

### Типы группировок

#### по столбцам: 

In [ ]:
tips.head()

In [ ]:
tips.groupby(['day','time']).tip.mean().unstack()

#### по уровням индекса

создаем копию данных и заново индексируем ее

In [ ]:
copy_tips = tips.copy()
copy_tips = copy_tips.set_index(['day', 'time'])
copy_tips

группировать можем по одному или нескольким уровням индекса, передавая соответствующие значения столбцов аргументу level

In [ ]:
copy_tips.groupby(level=['time']).sum()

In [ ]:
copy_tips.groupby(level = ['day', 'time']).mean()

#### с использованием функции 

In [ ]:
copy_tips.head()

In [ ]:
copy_tips.groupby(len, level='day').count()

In [ ]:
tips.day.value_counts()

#### с использованием массива 

массив случайных меток групп:

In [ ]:
np.random.seed(123)
rnd_array = np.random.choice(['first_group', 'second_group'], 
                             size=tips.shape[0],
                             p = [0.4, 0.6])
rnd_array[0:5]

передаем массив меток групп в метод groupby

In [ ]:
tips.groupby(rnd_array).count()

In [ ]:
93/(93+151)

#### бонус

тип данных:

In [ ]:
tips.dtypes

In [ ]:
group_tips = tips.groupby(tips.dtypes, axis=1)

In [ ]:
group_tips.size()

### Агрегирование групп

создаем объект groupby:

In [ ]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct']

проверка эквивалентности методов:

In [ ]:
group_tips.agg == group_tips.aggregate

собственная функция:

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

group_tips.agg([peak_to_peak])

комбинирование:

In [ ]:
group_tips.agg([peak_to_peak, 'mean'])

задаем имена столбцов:

In [ ]:
group_tips.agg([('delta_max_min', peak_to_peak), ('mean_value','mean')])

отдельные наборы функций для каждого столбца:

In [ ]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct', 'total_bill']

In [ ]:
group_tips.agg({'tip_pct':[('max_value', np.max),
                           ('min_value','min')],
                'total_bill': 'sum'})

### Преобразование групп

#### метод transform

<img src = '../images/transform.png' style='width: 900px;'/>

In [ ]:
df = pd.DataFrame({'Col1': ['A', 'B', 'C', 'C', 'B', 'B', 'A'],
                   'Col2': [1, 2, 3, 4, 2, 5, 3]})
df

In [ ]:
df['Col3'] = df.groupby('Col1').transform(sum)
df.sort_values('Col1')

объект groupby:

In [ ]:
group_tips = tips.groupby(['smoker'])['total_bill']

нормирование:

In [ ]:
norm = lambda x: (x - x.mean())/x.std()
group_tips.transform(norm)

In [ ]:
tips_copy = tips.copy()
tips_copy['total_bill_norm'] = group_tips.transform(norm)
tips_copy.head()

In [ ]:
tips_copy.groupby('smoker').total_bill_norm.agg(['mean', 'std'])

#### метод apply

функция:

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

применяем метод apply

In [ ]:
tips.groupby('smoker').apply(top)

### Исключение групп

создаем данные для наших примеров

In [ ]:
df = pd.DataFrame({'Label': list('AABCCC'),
                   'Values': [1, 2, 3, 4, np.nan, 8]})
df

удаляем группы с менее чем двумя возможными значениеми

In [ ]:
f = lambda x: x.Values.count() > 1
df.groupby('Label').filter(f)

удаляем группы, в которых есть пропуски

In [ ]:
f = lambda x: x.Values.isnull().sum() == 0
df.groupby('Label').filter(f)

### Сводная таблица

средние

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

вывод по строкам и столбцам

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

включение частичных итогов:

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

задаем функцию:

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=max, margins=True)